In [3]:
    Plain_hex=str(input("enter the Plain Hex (M): "))
    Key_hex = str(input("enter the Key Hex: "))
    Cipher_binary=""
    Key_binary = bin(int(Key_hex, 16))[2:].zfill(len(Key_hex) * 4)
    Plain_binary = bin(int(Plain_hex, 16))[2:].zfill(len(Plain_hex) * 4)
    
    PC1_mapping = [
    57, 49, 41, 33, 25, 17, 9,
    1, 58, 50, 42, 34, 26, 18,
    10, 2, 59, 51, 43, 35, 27,
    19, 11, 3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
    7, 62, 54, 46, 38, 30, 22,
    14, 6, 61, 53, 45, 37, 29,
    21, 13, 5, 28, 20, 12, 4
  ]
    permuted_key = ''.join([Key_binary[i - 1] for i in PC1_mapping])
    midpoint = len(permuted_key) // 2

    C = permuted_key[:midpoint]
    D = permuted_key[midpoint:]
    
    num_blocks = (len(Plain_binary) + 63) // 64
    padding_needed = 64 - (len(Plain_binary) % 64)
    blocks = [Plain_binary[i:i+64] for i in range(0, len(Plain_binary), 64)]
    if padding_needed != 64:
        blocks[-1] = '0' * padding_needed + blocks[-1]

    for i, block in enumerate(blocks):
        IP_mapping = [
                58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7
                ]
        IP = ''.join([block[i - 1] for i in IP_mapping])
        midpoint2 = len(IP) // 2
        L = IP[:midpoint2]
        R = IP[midpoint2:]

        for i in range(16):
            
            if i == 0 or i == 1 or i == 8 or i==15:
                C = C[1:] + C[0]
                D = D[1:] + D[0]
                CD = C + D
                PC2_mapping = [
                14, 17, 11, 24, 1, 5,
                3, 28, 15, 6, 21, 10,
                23, 19, 12, 4, 26, 8,
                16, 7, 27, 20, 13, 2,
                41, 52, 31, 37, 47, 55,
                30, 40, 51, 45, 33, 48,
                44, 49, 39, 56, 34, 53,
                46, 42, 50, 36, 29, 32
                ]
                K = ''.join([CD[i - 1] for i in PC2_mapping])
                
                Z=L
                L=R
                
                E_table = [
                32, 1, 2, 3, 4, 5,
                4, 5, 6, 7, 8, 9,
                8, 9, 10, 11, 12, 13,
                12, 13, 14, 15, 16, 17,
                16, 17, 18, 19, 20, 21,
                20, 21, 22, 23, 24, 25,
                24, 25, 26, 27, 28, 29,
                28, 29, 30, 31, 32, 1
                ]

                ER = ''.join([R[i - 1] for i in E_table])
                K_ER = ''.join([str(int(K[i]) ^ int(ER[i])) for i in range(len(K))])
                
                S_boxes = [
                    # S1
                    [
                        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
                        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
                        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
                    ],
                    # S2
                    [
                        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
                        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
                        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
                        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
                    ],
                    # S3
                    [
                        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
                        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
                        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
                        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
                    ],
                    # S4
                    [
                        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
                        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
                        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
                        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
                    ],
                    # S5
                    [
                        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
                        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
                        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
                        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
                    ],
                    # S6
                    [
                        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
                        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
                        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
                        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
                    ],
                    # S7
                    [
                        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
                        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
                        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
                        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
                    ],
                    # S8
                    [
                        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
                        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
                        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
                        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
                    ]
                ]
                blocks_K_ER = [K_ER[i:i+6] for i in range(0, len(K_ER), 6)]
                substitutions = ""
                for i, block_K_ER in enumerate(blocks_K_ER):
                    S_box = S_boxes[i]
                    row = int(block_K_ER[0] + block_K_ER[5], 2)
                    col = int(block_K_ER[1:5], 2)
                    substitution_value = format(S_box[row][col], '04b')
                    substitutions += substitution_value

                P_table = [
                16, 7, 20, 21,
                29, 12, 28, 17,
                1, 15, 23, 26,
                5, 18, 31, 10,
                2, 8, 24, 14,
                32, 27, 3, 9,
                19, 13, 30, 6,
                22, 11, 4, 25
                ]
                f = ''.join(substitutions[i - 1] for i in P_table)
                R = ''.join([str(int(Z[i]) ^ int(f[i])) for i in range(len(Z))])
               

            else:
                C = C[2:] + C[:2]
                D = D[2:] + D[:2]
                CD = C + D
                
                PC2_mapping = [
                14, 17, 11, 24, 1, 5,
                3, 28, 15, 6, 21, 10,
                23, 19, 12, 4, 26, 8,
                16, 7, 27, 20, 13, 2,
                41, 52, 31, 37, 47, 55,
                30, 40, 51, 45, 33, 48,
                44, 49, 39, 56, 34, 53,
                46, 42, 50, 36, 29, 32
                ]
                K = ''.join([CD[i - 1] for i in PC2_mapping])
                
                Z=L
                L=R
                
                E_table = [
                32, 1, 2, 3, 4, 5,
                4, 5, 6, 7, 8, 9,
                8, 9, 10, 11, 12, 13,
                12, 13, 14, 15, 16, 17,
                16, 17, 18, 19, 20, 21,
                20, 21, 22, 23, 24, 25,
                24, 25, 26, 27, 28, 29,
                28, 29, 30, 31, 32, 1
                ]

                ER = ''.join([R[i - 1] for i in E_table])
                K_ER = ''.join([str(int(K[i]) ^ int(ER[i])) for i in range(len(K))])
                
                S_boxes = [
                    # S1
                    [
                        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
                        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
                        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
                    ],
                    # S2
                    [
                        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
                        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
                        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
                        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
                    ],
                    # S3
                    [
                        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
                        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
                        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
                        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
                    ],
                    # S4
                    [
                        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
                        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
                        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
                        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
                    ],
                    # S5
                    [
                        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
                        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
                        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
                        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
                    ],
                    # S6
                    [
                        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
                        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
                        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
                        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
                    ],
                    # S7
                    [
                        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
                        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
                        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
                        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
                    ],
                    # S8
                    [
                        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
                        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
                        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
                        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
                    ]
                ]
                blocks_K_ER = [K_ER[i:i+6] for i in range(0, len(K_ER), 6)]
                substitutions = ""
                for i, block_K_ER in enumerate(blocks_K_ER):
                    S_box = S_boxes[i]
                    row = int(block_K_ER[0] + block_K_ER[5], 2)
                    col = int(block_K_ER[1:5], 2)
                    substitution_value = format(S_box[row][col], '04b')
                    substitutions += substitution_value

                P_table = [
                16, 7, 20, 21,
                29, 12, 28, 17,
                1, 15, 23, 26,
                5, 18, 31, 10,
                2, 8, 24, 14,
                32, 27, 3, 9,
                19, 13, 30, 6,
                22, 11, 4, 25
                ]
                f = ''.join(substitutions[i - 1] for i in P_table)
                R = ''.join([str(int(Z[i]) ^ int(f[i])) for i in range(len(Z))])
                

        print("L16 :",L)
        print("R16 :",R)
        RL=R+L
        print("R16L16 :",RL)
        IP_inv_table = [
        40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25
        ]
        RL_IP_inv = ''.join(RL[i - 1] for i in IP_inv_table)
        print("Cipher as binary (IP) :", RL_IP_inv)
        Cipher_binary=Cipher_binary+RL_IP_inv
    
    chunks = [Cipher_binary[i:i+8] for i in range(0, len(Cipher_binary), 8)]

    characters = [chr(int(chunk, 2)) for chunk in chunks]

    Cipher_text = ''.join(characters)
    
    RL_IP_inv_int = int(RL_IP_inv, 2)
    Cipher_hex = hex(RL_IP_inv_int)[2:]
    
    print("Cipher as hex (C):", Cipher_hex)
    print("Cipher as text :", Cipher_text)
    

enter the Plain Hex (M): 0123456789ABCDEF
enter the Key Hex: 133457799BBCDFF1
L16 : 01000011010000100011001000110100
R16 : 00001010010011001101100110010101
R16L16 : 0000101001001100110110011001010101000011010000100011001000110100
Cipher as binary (IP) : 1000010111101000000100110101010000001111000010101011010000000101
Cipher as hex (C): 85e813540f0ab405
Cipher as text : èT
´
